# Week 6: Building Agents with LangGraph

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Digital-AI-Finance/agentic-artificial-intelligence/blob/main/L06_Agent_Frameworks/L06_LangGraph_Agent.ipynb)

This notebook implements a stateful agent using LangGraph with:
- Graph-based state machine architecture
- Tool calling with conditional routing
- Checkpointing for state persistence
- Human-in-the-loop interrupts

In [ ]:
# Colab setup
import sys
if 'google.colab' in sys.modules:
    !pip install -q langgraph langchain-openai langchain-anthropic python-dotenv
    from google.colab import userdata
    import os
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
import os
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
import operator

# Load environment
from dotenv import load_dotenv
load_dotenv()

print("LangGraph environment ready")

## 1. Define Tools

Tools are functions the agent can call to interact with the environment.

In [ ]:
@tool
def calculator(expression: str) -> str:
    """Evaluate a mathematical expression. Use for calculations."""
    try:
        # Safe eval for basic math
        allowed = set('0123456789+-*/.() ')
        if not all(c in allowed for c in expression):
            return "Error: Invalid characters in expression"
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def search_knowledge(query: str) -> str:
    """Search the knowledge base for information."""
    # Simulated knowledge base
    knowledge = {
        "langgraph": "LangGraph is a library for building stateful, multi-actor applications with LLMs.",
        "autogen": "AutoGen is a framework for building multi-agent conversational systems.",
        "react": "ReAct combines reasoning and acting in language model agents.",
    }
    query_lower = query.lower()
    for key, value in knowledge.items():
        if key in query_lower:
            return value
    return "No relevant information found."

tools = [calculator, search_knowledge]
print(f"Defined {len(tools)} tools: {[t.name for t in tools]}")

## 2. Define State Schema

The state schema defines what data flows through the graph.

In [ ]:
class AgentState(TypedDict):
    """State schema for the agent."""
    messages: Annotated[Sequence[BaseMessage], operator.add]
    iteration_count: int

# Messages use the 'add' reducer - new messages are appended
# iteration_count tracks how many agent loops have executed
print("State schema defined with messages and iteration_count")

## 3. Define Graph Nodes

Nodes are functions that take state and return state updates.

In [ ]:
# Initialize LLM with tool binding
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)

def agent_node(state: AgentState) -> dict:
    """The agent node - calls LLM to decide next action."""
    messages = state["messages"]
    iteration = state.get("iteration_count", 0)
    
    # Call LLM
    response = llm_with_tools.invoke(messages)
    
    return {
        "messages": [response],
        "iteration_count": iteration + 1
    }

# Tool node handles tool execution
tool_node = ToolNode(tools)

print("Agent and tool nodes defined")

## 4. Define Routing Logic

Conditional edges route based on the agent's decision.

In [ ]:
def should_continue(state: AgentState) -> str:
    """Determine if agent should continue or end."""
    messages = state["messages"]
    last_message = messages[-1]
    iteration = state.get("iteration_count", 0)
    
    # Safety limit on iterations
    if iteration >= 10:
        return "end"
    
    # Check if LLM wants to call tools
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    
    return "end"

print("Routing logic defined")

## 5. Build the Graph

In [ ]:
# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

# Set entry point
workflow.set_entry_point("agent")

# Add conditional edges
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        "end": END
    }
)

# Tools always return to agent
workflow.add_edge("tools", "agent")

# Compile with checkpointing
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

print("Graph compiled successfully")
print("Nodes:", list(app.nodes.keys()))

## 6. Run the Agent

In [ ]:
def run_agent(query: str, thread_id: str = "default"):
    """Run the agent with a query."""
    config = {"configurable": {"thread_id": thread_id}}
    
    initial_state = {
        "messages": [HumanMessage(content=query)],
        "iteration_count": 0
    }
    
    print(f"\n{'='*50}")
    print(f"Query: {query}")
    print(f"{'='*50}")
    
    # Stream events
    for event in app.stream(initial_state, config):
        for node_name, output in event.items():
            print(f"\n[{node_name}]")
            if "messages" in output:
                for msg in output["messages"]:
                    if hasattr(msg, "content") and msg.content:
                        print(f"  Content: {msg.content[:200]}..." if len(str(msg.content)) > 200 else f"  Content: {msg.content}")
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        for tc in msg.tool_calls:
                            print(f"  Tool Call: {tc['name']}({tc['args']})")
    
    # Get final state
    final_state = app.get_state(config)
    print(f"\nIterations: {final_state.values.get('iteration_count', 'N/A')}")
    return final_state

In [ ]:
# Test with calculation
result = run_agent("What is 25 * 17 + 123?")

In [ ]:
# Test with knowledge search
result = run_agent("What is LangGraph and how does it differ from other frameworks?")

In [ ]:
# Test multi-step reasoning
result = run_agent("First, search for what ReAct is. Then calculate 2^10.")

## 7. Examine Checkpoints

LangGraph persists state at each step, enabling debugging and resumption.

In [ ]:
# Get state history
config = {"configurable": {"thread_id": "default"}}
states = list(app.get_state_history(config))

print(f"\nState history ({len(states)} checkpoints):")
for i, state in enumerate(states[:5]):  # Show last 5
    print(f"  {i+1}. Iteration: {state.values.get('iteration_count', 'N/A')}, "
          f"Messages: {len(state.values.get('messages', []))}")

## Summary

In this notebook, we implemented:
1. **State Schema**: TypedDict with message history and iteration count
2. **Nodes**: Agent node (LLM) and tool node (execution)
3. **Edges**: Conditional routing based on tool calls
4. **Checkpointing**: MemorySaver for state persistence

Key patterns:
- Graph cycles enable iterative refinement
- State reducers control how updates merge
- Checkpoints enable debugging and resumption